# Using MLflow REST with Azure ML

The MLflow REST API allows you to create, list, and get experiments and runs, and log parameters, metrics, and artifacts. The API is hosted under the `/api/2.0/mlflow` route on for Azure ML MLflow tracking URL.

In [1]:
import requests

## Getting an authorization token

As with any Azure service, you need an authentication token to access the MLflow REST API. The following example shows how to get an authentication token using a service principal. You will need the tenant ID, client ID and client secret associated with the service principal. To create a service principal you can follow the steps at [Use the portal to create an Azure AD application and service principal that can access resources](https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal).

In [2]:
tenant_id = "<>"
client_id = "<>"
client_secret = "<>"
grant_type = "client_credentials"
resource_scope = "https://management.azure.com"

The request URL is as follows:

In [10]:
mso_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token?"

The service principal information is included inside the body:

In [15]:
mso_body = f"grant_type={grant_type}&client_id={client_id}&client_secret={client_secret}&resource={resource_scope}"

Making the request and getting the token back:

In [12]:
auth_response = requests.post(url=mso_url, data=mso_body)

In [13]:
token_data = auth_response.json()

> Token is returned in the field `access_token`.

## Querying the MLflow API

You can get the MLflow API from the Azure portal, or you can construct it manually as follows:

In [ ]:
subscription_id = "<>"
resource_group = "<>"
workspace_name = "<>"
location = "eastus"

In [27]:
azure_ml_mlflow_base = f"https://{location}.api.azureml.ms/mlflow/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}"
mlflow_api_path = "api/2.0/mlflow"

> You can get this URL following this steps: Navigate to Azure ML Studio and select the workspace you are working on > Click on the name of the workspace at the upper right corner of the page. > Click “View all properties in Azure Portal” on the pane popup. > Copy the MLflow tracking URI value from the properties section. Then replace the protocol from `azureml` to `https`.

Construct the request headers including the authentication token you just got:

In [20]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"{token_data['token_type']} {token_data['access_token']}",
}

Construct the request URL as indicated at https://mlflow.org/docs/latest/rest-api.html. For example, the following URL list all the experiments in the workspace:

In [31]:
response = requests.get(
    f"{azure_ml_mlflow_base}/{mlflow_api_path}/experiments/list", headers=headers
)

In the following example, we are retrieving the experiments and parsing the results into a Pandas DataFrame for visualization:

In [33]:
data = response.json()

In [36]:
import pandas as pd

In [37]:
pd.DataFrame(data["experiments"])

,experiment_id,name,lifecycle_stage,creation_time
0,ae8d6548-ffed-4de3-97a6-e035a6a63843,gas-mixture-ppc,active,1643139376402
1,2252b012-21a5-4fc5-bfd7-1c3163a31c9a,sk-learn-random-forest,active,1643644632553
2,a806d448-b928-406d-835c-281a421a598c,Default,active,1643656927962
3,49fca8fc-1cb7-45a6-9b97-a5881162f0c0,SklearnLinearRegression,active,1643745668603
4,392bdd02-5d08-49be-90f3-a78e81d7aa71,mlflow-adult-classifier,active,1644443878352
...,...,...,...,...
56,b4ae0af5-3f02-4e6c-9031-bc6a1291aa9f,demo-hr-employee,active,1586196983332
57,ba4c66ed-6bcd-401f-9991-b1ebb4023c76,prediction-nyc-energy,active,1586198805982
58,9d663c97-b8de-4446-b307-fa17d4d26cc8,Hyperdrive_sample,active,1590524844101
59,ff1b689b-1d70-47e1-89f2-d767573f8045,credit-card-fraud-detection,active,1597016129774
